In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = c8fa359
molecule = {'h2o': 'atmpro', 'co2': 0.0004, 'n2o': 3.2e-07}
band = [3]
atmpro = trp
tsfc = 300


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band3a
  atmpro = mls
  band = 3a
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 0
  molecule = h2o
  ng_adju = [0, 0]
  ng_refs = [2, 6]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(10, 250), (600, 250)]
  tsfc = 294
  vmax = 620
  vmin = 540
  w_diffuse = [(1.9, 1.7), (1.4, 1.4, 1.4, 1.55, 1.6, 1.66)]
  wgt = [(0.7, 0.7), (0.7, 0.5, 0.5, 0.5, 0.5, 0.5)]
o co2 band3a
  atmpro = mls
  band = 3a
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 6.375563e-24
  molecule = co2
  ng_adju = [0, 0, 0]
  ng_refs = [3, 2, 4]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (10, 250), (500, 250)]
  tsfc = 294
  vmax = 620
  vmin = 540
  w_diffuse = [(1.6, 1.6, 1.7), (1.75, 1.75), (1.55, 1.55, 1.6, 1.85)]
  wgt = [(0.7, 0.3, 0.7), (0

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`       
            layer    coolrg                layer coolrg
pressure                                               
0.000312        1  0.393544                    1    0.0
0.000750        2  0.393826                    2    0.0
0.001052        3  0.407652                    3    0.0
0.001476        4  0.419989                    4    0.0
0.002070        5  0.430245                    5    0.0
0.002904        6  0.437232                    6    0.0
0.004074        7  0.440234                    7    0.0
0.005714        8  0.437915                    8    0.0
0.008015        9  0.428933                    9    0.0
0.011243       10  0.411771                   10    0.0
0.015771       11  0.384794                   11    0.0
0.022122       12  0.346205                   12    0.0
0.031031       13  0.293922                   13    0.0
0.043528       14  0.229687                   14    0.0
0.061057       15  0.208438                   15    0.0
0.085645       16  0.257563                   16    0.0
0.120136       17  0.321923                   17    0.0
0.168516       18  0.371046                   18    0.0
0.236378       19  0.414875                   19    0.0
0.331549       20  0.469674                   20    0.0
0.465100       21  0.560351                   21    0.0
0.652400       22  0.699836                   22    0.0
0.915100       23  0.704428                   23    0.0
1.283650       24  0.598948                   24    0.0
1.800600       25  0.507972                   25    0.0
2.525700       26  0.412678                   26    0.0
3.542800       27  0.336798                   27    0.0
4.969550       28  0.276840                   28    0.0
6.970850       29  0.226215                   29    0.0
9.778100       30  0.182400                   30    0.0
13.715850      31  0.144233                   31    0.0
19.239350      32  0.111211                   32    0.0
26.987250      33  0.083351                   33    0.0
37.855300      34  0.061138                   34    0.0
53.100050      35  0.030482                   35    0.0
73.887500      36 -0.005806                   36    0.0
97.662500      37 -0.028726                   37    0.0
121.437500     38 -0.025035                   38    0.0
145.212500     39 -0.004890                   39    0.0
168.987500     40  0.009253                   40    0.0
192.762500     41  0.023001                   41    0.0
216.537500     42  0.039931                   42    0.0
240.312500     43  0.060557                   43    0.0
264.087500     44  0.084200                   44    0.0
287.862500     45  0.109394                   45    0.0
311.637500     46  0.135228                   46    0.0
335.412500     47  0.161308                   47    0.0
359.187500     48  0.186203                   48    0.0
382.962500     49  0.204963                   49    0.0
406.737500     50  0.225413                   50    0.0
430.512500     51  0.246860                   51    0.0
454.287500     52  0.268816                   52    0.0
478.062500     53  0.290744                   53    0.0
501.837500     54  0.311660                   54    0.0
525.612500     55  0.331270                   55    0.0
549.387500     56  0.348548                   56    0.0
573.162500     57  0.363046                   57    0.0
596.937500     58  0.374588                   58    0.0
620.712500     59  0.382803                   59    0.0
644.487500     60  0.388297                   60    0.0
668.262500     61  0.390868                   61    0.0
692.037500     62  0.388423                   62    0.0
715.812500     63  0.377532                   63    0.0
739.587500     64  0.372229                   64    0.0
763.362500     65  0.345107                   65    0.0
787.137500     66  0.307700                   66    0.0
810.912500     67  0.268947                   67    0.0
834.687500     68  0.226845                   68    0.0
858.462500     69  0.194086           

# Fluxes by Layer

CRD                          CLIRAD               
                        flug       fldg      fnetg      flug fldg     fnetg
pressure    level                                                          
0.000000    1     -25.153740   0.000000 -25.153740 -38.49173  0.0 -38.49173
0.000624    2     -25.153758   0.000048 -25.153710 -38.49173  0.0 -38.49173
0.000876    3     -25.153766   0.000067 -25.153699 -38.49173  0.0 -38.49173
0.001229    4     -25.153776   0.000095 -25.153682 -38.49173  0.0 -38.49173
0.001723    5     -25.153791   0.000134 -25.153657 -38.49173  0.0 -38.49173
0.002417    6     -25.153811   0.000189 -25.153622 -38.49173  0.0 -38.49173
0.003391    7     -25.153839   0.000267 -25.153571 -38.49173  0.0 -38.49173
0.004757    8     -25.153877   0.000377 -25.153500 -38.49173  0.0 -38.49173
0.006672    9     -25.153930   0.000529 -25.153401 -38.49173  0.0 -38.49173
0.009359    10    -25.154004   0.000740 -25.153264 -38.49173  0.0 -38.49173
0.013128    11    -25.154107   0.001027 -25.153080 -38.49173  0.0 -38.49173
0.018415    12    -25.154254   0.001414 -25.152839 -38.49173  0.0 -38.49173
0.025830    13    -25.154462   0.001927 -25.152535 -38.49173  0.0 -38.49173
0.036232    14    -25.154767   0.002594 -25.152173 -38.49173  0.0 -38.49173
0.050823    15    -25.155225   0.003449 -25.151776 -38.49173  0.0 -38.49173
0.071291    16    -25.155898   0.004627 -25.151271 -38.49173  0.0 -38.49173
0.100000    17    -25.156815   0.006420 -25.150395 -38.49173  0.0 -38.49173
0.140271    18    -25.157998   0.009139 -25.148859 -38.49173  0.0 -38.49173
0.196760    19    -25.159464   0.013088 -25.146376 -38.49173  0.0 -38.49173
0.275997    20    -25.161175   0.018693 -25.142482 -38.49173  0.0 -38.49173
0.387100    21    -25.162935   0.026634 -25.136301 -38.49173  0.0 -38.49173
0.543100    22    -25.164042   0.038096 -25.125946 -38.49173  0.0 -38.49173
0.761700    23    -25.162384   0.054560 -25.107824 -38.49173  0.0 -38.49173
1.068500    24    -25.157088   0.074865 -25.082223 -38.49173  0.0 -38.49173
1.498800    25    -25.149188   0.097495 -25.051693 -38.49173  0.0 -38.49173
2.102400    26    -25.138748   0.123375 -25.015373 -38.49173  0.0 -38.49173
2.949000    27    -25.126572   0.152584 -24.973987 -38.49173  0.0 -38.49173
4.136600    28    -25.112872   0.186266 -24.926607 -38.49173  0.0 -38.49173
5.802500    29    -25.097565   0.225589 -24.871976 -38.49173  0.0 -38.49173
8.139200    30    -25.080739   0.271379 -24.809359 -38.49173  0.0 -38.49173
11.417000   31    -25.062790   0.324253 -24.738538 -38.49173  0.0 -38.49173
16.014700   32    -25.044626   0.384642 -24.659984 -38.49173  0.0 -38.49173
22.464000   33    -25.027922   0.452900 -24.575022 -38.49173  0.0 -38.49173
31.510500   34    -25.015168   0.529467 -24.485701 -38.49173  0.0 -38.49173
44.200100   35    -25.007926   0.614125 -24.393801 -38.49173  0.0 -38.49173
62.000000   36    -25.023453   0.693925 -24.329529 -38.49173  0.0 -38.49173
85.775000   37    -25.101329   0.755448 -24.345880 -38.49173  0.0 -38.49173
109.550000  38    -25.236454   0.809672 -24.426782 -38.49173  0.0 -38.49173
133.325000  39    -25.410049   0.912762 -24.497287 -38.49173  0.0 -38.49173
157.100000  40    -25.598522   1.087464 -24.511058 -38.49173  0.0 -38.49173
180.875000  41    -25.798937   1.313939 -24.484998 -38.49173  0.0 -38.49173
204.650000  42    -26.009904   1.589684 -24.420219 -38.49173  0.0 -38.49173
228.425000  43    -26.233245   1.925483 -24.307762 -38.49173  0.0 -38.49173
252.200000  44    -26.469663   2.332448 -24.137215 -38.49173  0.0 -38.49173
275.975000  45    -26.720428   2.820348 -23.900080 -38.49173  0.0 -38.49173
299.750000  46    -26.987050   3.395059 -23.591991 -38.49173  0.0 -38.49173
323.525000  47    -27.271151   4.060004 -23.211147 -38.49173  0.0 -38.49173
347.300000  48    -27.574500   4.817648 -22.756853 -38.49173  0.0 -38.49173
371.075000  49    -27.898024   5.665578 -22.232446 -38.49173  0.0 -38.49173
394.850000  50    -28.236603   6.581397 -21.655206 -38.49173  0.0 -38.491

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')